In [1]:
import numpy as np

In [2]:
D= [[3,3],[1,1],[-1,0],[2,2],[-2,2],[-2,-2],[0,-2]]

In [3]:
index = len(D)
index

7

In [4]:
cost_similarity = [[0 for i in range(index)] for j in range( index)]
cost_similarity

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]

In [5]:
for i in range(index):
  for j in range(index):
    dot_product = np.dot(D[i],D[j])
    l2Norm =np.linalg.norm(D[i])*np.linalg.norm(D[j])
    cost_similarity[i][j] =dot_product/l2Norm

In [6]:
cost_similarity

[[1.0000000000000002,
  1.0,
  -0.7071067811865476,
  1.0,
  0.0,
  -1.0,
  -0.7071067811865476],
 [1.0,
  0.9999999999999998,
  -0.7071067811865475,
  0.9999999999999998,
  0.0,
  -0.9999999999999998,
  -0.7071067811865475],
 [-0.7071067811865476,
  -0.7071067811865475,
  1.0,
  -0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475,
  0.0],
 [1.0,
  0.9999999999999998,
  -0.7071067811865475,
  0.9999999999999998,
  0.0,
  -0.9999999999999998,
  -0.7071067811865475],
 [0.0,
  0.0,
  0.7071067811865475,
  0.0,
  0.9999999999999998,
  0.0,
  -0.7071067811865475],
 [-1.0,
  -0.9999999999999998,
  0.7071067811865475,
  -0.9999999999999998,
  0.0,
  0.9999999999999998,
  0.7071067811865475],
 [-0.7071067811865476,
  -0.7071067811865475,
  0.0,
  -0.7071067811865475,
  -0.7071067811865475,
  0.7071067811865475,
  1.0]]

In [7]:
from scipy.spatial.distance import pdist, squareform, cdist


In [8]:
dist = pdist(D, 'euclidean')
dist

array([2.82842712, 5.        , 1.41421356, 5.09901951, 7.07106781,
       5.83095189, 2.23606798, 1.41421356, 3.16227766, 4.24264069,
       3.16227766, 3.60555128, 2.23606798, 2.23606798, 2.23606798,
       4.        , 5.65685425, 4.47213595, 4.        , 4.47213595,
       2.        ])

In [9]:
D = np.array(D)

In [10]:
D

array([[ 3,  3],
       [ 1,  1],
       [-1,  0],
       [ 2,  2],
       [-2,  2],
       [-2, -2],
       [ 0, -2]])

In [11]:
def k_mean(centroids, D, max_iter, convergence, metric):
  """
    Calculate the k_mean with-in limited iteration and convergence threadhold.

    D: array of data points
    centroids: array of cluster centroids
    max_iter: max iteration
    convergence: threadhold to decide if the centroid converged
    metric: e for euclidean, c for cosine similarity
    """
  k = centroids.shape[0]
  iteration = 0
  while iteration < max_iter:
    if metric == "e":

      distances = cdist(centroids, D, metric='euclidean')
    else:

      cosine_distances = cdist(centroids, D, metric='cosine')
      # Since the cosine distance is 1 - cosine similarity, we can recover the similarities:
      distances = 1 - cosine_distances

    closest_centroids = np.argmin(distances, axis=0)
    clusters = [[] for _ in range(k)]
    for point_idx in range(len(D)):
      centroid_idx = closest_centroids[point_idx]  # this is the correct centroid index
      clusters[centroid_idx].append(D[point_idx])
    #update centroid
    cluster_means = []

    # Re-calculate centroids
    new_centroids =[]
    empty_clusters = False

    for idx, cluster in enumerate(clusters):
        if len(cluster) == 0:  # cluster is empty
            empty_clusters = True
            # One approach to handle empty clusters: reassign to random point from the data
            new_centroids.append(D[np.random.choice(D.shape[0]), :])
        else:
            new_centroids.append(np.mean(cluster, axis=0))

    new_centroids = np.array(new_centroids)

    # If we reassigned any centroids, we skip the convergence check for this iteration
    if not empty_clusters:
        differences = np.linalg.norm(new_centroids - centroids, axis=1)
        if np.max(differences) < convergence:
            print("Convergence achieved.")
            break

    centroids = new_centroids
    iteration += 1


  print("Iteration: ", iteration)
  print("Clusters: \n", clusters)
  print("Centroids: \n", centroids)
  return iteration, clusters, centroids

In [60]:
def simplified_silhouette(D, clusters, centroids):
    """
    Calculate the simplified silhouette score.

    D: array of data points
    clusters: list of arrays with data points associated with each cluster
    centroids: array of cluster centroids
    """
    all_silhouettes = []

    # Iterate through each data point in D to calculate silhouette scores
    for point in D:
        # Find the cluster to which the point belongs
        distances_to_centroids = np.linalg.norm(centroids - point, axis=1)
        own_cluster_idx = np.argmin(distances_to_centroids)

        # a is the distance from point to its own cluster's centroid
        a = distances_to_centroids[own_cluster_idx]

        # Calculate b, the average distance of point to the centroids of the other clusters
        sum_other_distances = np.sum(distances_to_centroids) - a
        # We subtract 1 to exclude the own cluster's centroid from the count
        b = sum_other_distances / (len(centroids) - 1)

        # Calculate the silhouette score for the current point
        silhouette = (b - a) / max(a, b) if max(a, b) > 0 else 0  # handle the edge case to avoid division by zero
        all_silhouettes.append(silhouette)

    # Sum all silhouette scores
    total_silhouette = np.sum(all_silhouettes)
    return total_silhouette



In [13]:
def calculate_wss(clusters, centroids):
    total_wss = 0.0

    # For each cluster and its corresponding centroid
    for idx, cluster in enumerate(clusters):
        if len(cluster) == 0:  # Skip if cluster is empty
            continue
        # Calculate squared distances from points to the centroid
        squared_distances = np.linalg.norm(cluster - centroids[idx], axis=1) ** 2
        # Sum up squared distances for this cluster
        total_wss += np.sum(squared_distances)

    return total_wss


In [57]:
centroids = np.array([[3,3],[2,2]])

In [58]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "e")

Convergence achieved.
Iteration:  3
Clusters: 
 [[array([3, 3]), array([1, 1]), array([2, 2])], [array([-1,  0]), array([-2,  2]), array([-2, -2]), array([ 0, -2])]]
Centroids: 
 [[ 2.    2.  ]
 [-1.25 -0.5 ]]


In [61]:
simplified_silhouette(D, clusters, final_centroid)

4.679526784405805

In [17]:
calculate_wss(clusters, final_centroid)

17.750000000000004

In [18]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "c")

Iteration:  100
Clusters: 
 [[array([3, 3]), array([1, 1]), array([2, 2])], [array([-1,  0]), array([-2,  2]), array([-2, -2]), array([ 0, -2])]]
Centroids: 
 [[ 2.    2.  ]
 [-1.25 -0.5 ]]


In [19]:
simplified_silhouette(D, clusters, final_centroid)

4.679526784405805

In [20]:
calculate_wss(clusters, final_centroid)

17.750000000000004

In [21]:
centroids = np.array([[3,3],[2,2],[-2,-2]])

In [22]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "e")

Convergence achieved.
Iteration:  2
Clusters: 
 [[array([3, 3]), array([2, 2])], [array([1, 1]), array([-2,  2])], [array([-1,  0]), array([-2, -2]), array([ 0, -2])]]
Centroids: 
 [[ 2.5         2.5       ]
 [-0.5         1.5       ]
 [-1.         -1.33333333]]


In [23]:
simplified_silhouette(D, clusters, final_centroid)

4.683092063446775

In [24]:
calculate_wss(clusters, final_centroid)

10.666666666666668

In [25]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "c")

Iteration:  100
Clusters: 
 [[array([-2,  2])], [array([3, 3]), array([1, 1]), array([2, 2])], [array([-1,  0]), array([-2, -2]), array([ 0, -2])]]
Centroids: 
 [[-2.          2.        ]
 [ 2.          2.        ]
 [-1.         -1.33333333]]


In [26]:
simplified_silhouette(D, clusters, final_centroid)

5.315032070682842

In [27]:
calculate_wss(clusters, final_centroid)

8.666666666666668

In [28]:
centroids = np.array([[3,3],[2,2],[0,-2],[-2,-2]])

In [29]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "e")

Convergence achieved.
Iteration:  2
Clusters: 
 [[array([3, 3]), array([2, 2])], [array([1, 1]), array([-2,  2])], [array([-1,  0]), array([ 0, -2])], [array([-2, -2])]]
Centroids: 
 [[ 2.5  2.5]
 [-0.5  1.5]
 [-0.5 -1. ]
 [-2.  -2. ]]


In [30]:
simplified_silhouette(D, clusters, final_centroid)

5.032398567152775

In [31]:
calculate_wss(clusters, final_centroid)

8.500000000000002

In [32]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "c")

Iteration:  100
Clusters: 
 [[array([-2,  2])], [array([3, 3]), array([1, 1]), array([2, 2])], [array([ 0, -2])], [array([-1,  0]), array([-2, -2])]]
Centroids: 
 [[-2.   2. ]
 [ 2.   2. ]
 [ 0.  -2. ]
 [-1.5 -1. ]]


In [33]:
simplified_silhouette(D, clusters, final_centroid)

5.601371778656567

In [34]:
calculate_wss(clusters, final_centroid)

6.500000000000002

# Try different initialization

## Try k=2

In [91]:
centroids = np.array([[-10,2.5],[2.5,-1.5]])

In [92]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "e")

Convergence achieved.
Iteration:  2
Clusters: 
 [[array([-2,  2])], [array([3, 3]), array([1, 1]), array([-1,  0]), array([2, 2]), array([-2, -2]), array([ 0, -2])]]
Centroids: 
 [[-2.          2.        ]
 [ 0.5         0.33333333]]


In [93]:
simplified_silhouette(D, clusters, final_centroid)

3.383342274686864

In [94]:
calculate_wss(clusters, final_centroid)

38.833333333333336

In [80]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "c")

Iteration:  100
Clusters: 
 [[array([3, 3]), array([1, 1]), array([2, 2])], [array([-1,  0]), array([-2,  2]), array([-2, -2]), array([ 0, -2])]]
Centroids: 
 [[ 2.    2.  ]
 [-1.25 -0.5 ]]


In [81]:
simplified_silhouette(D, clusters, final_centroid)

4.679526784405805

In [82]:
calculate_wss(clusters, final_centroid)

17.750000000000004

## Try k=3

In [42]:
centroids = np.array([[2,1],[8,3],[-2,-6]])

In [43]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "e")

Convergence achieved.
Iteration:  2
Clusters: 
 [[array([-1,  0]), array([-2,  2])], [array([3, 3]), array([1, 1]), array([2, 2])], [array([-2, -2]), array([ 0, -2])]]
Centroids: 
 [[-1.5  1. ]
 [ 2.   2. ]
 [-1.  -2. ]]


In [44]:
simplified_silhouette(D, clusters, final_centroid)

5.127393271728211

In [45]:
calculate_wss(clusters, final_centroid)

8.500000000000002

In [46]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "c")

Iteration:  100
Clusters: 
 [[array([-2,  2])], [array([-1,  0]), array([-2, -2]), array([ 0, -2])], [array([3, 3]), array([1, 1]), array([2, 2])]]
Centroids: 
 [[-2.          2.        ]
 [-1.         -1.33333333]
 [ 2.          2.        ]]


In [47]:
simplified_silhouette(D, clusters, final_centroid)

5.315032070682842

In [48]:
calculate_wss(clusters, final_centroid)

8.666666666666668

## Try k=4

In [49]:
centroids = np.array([[2,1],[8,3],[-2,-6],[9,0.5]])

In [50]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "e")

Convergence achieved.
Iteration:  2
Clusters: 
 [[array([3, 3]), array([1, 1]), array([2, 2])], [array([-2,  2])], [array([-2, -2]), array([ 0, -2])], [array([-1,  0])]]
Centroids: 
 [[ 2.  2.]
 [-2.  2.]
 [-1. -2.]
 [-1.  0.]]


In [51]:
simplified_silhouette(D, clusters, final_centroid)

5.751124139825558

In [52]:
calculate_wss(clusters, final_centroid)

6.000000000000001

In [53]:
iteration, clusters, final_centroid = k_mean(centroids, D, 100, 0.1, "c")

Iteration:  100
Clusters: 
 [[array([-2,  2])], [array([3, 3]), array([1, 1]), array([2, 2])], [array([ 0, -2])], [array([-1,  0]), array([-2, -2])]]
Centroids: 
 [[-2.   2. ]
 [ 2.   2. ]
 [ 0.  -2. ]
 [-1.5 -1. ]]


In [54]:
simplified_silhouette(D, clusters, final_centroid)

5.601371778656567

In [55]:
calculate_wss(clusters, final_centroid)

6.500000000000002